In [1]:
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import tqdm
import copy
import pandas as pd
import statistics
import datetime

In [2]:
class Pandemic_Network():
    
    def __init__(self, network_type: str, nodes: int, pandemicprob: float, reduced_prob: float ,mitigation_proportion: float, sicknode = 0, SW_connections = 3, edge_randomness = 0.2, SF_k = 1, plots = False, aspl = False, TTR = 15):
        '''
        sicknodes - number of sick nodes to begin with
        '''

        if network_type == 'Ring': # default is a ring network
            self.g = nx.watts_strogatz_graph(n = nodes, k = SW_connections, p = 0, seed=None)
        elif network_type == 'Small World':
            self.g = nx.watts_strogatz_graph(n = nodes, k = SW_connections, p = edge_randomness, seed=None)
        elif network_type == 'Scale Free':
            self.g = nx.barabasi_albert_graph(n = nodes, m = SF_k, seed=None, initial_graph=None) # m = Number of edges to attach from a new node to existing nodes
        else:
            self.g = nx.erdos_renyi_graph(n = nodes, p = edge_randomness, seed=None, directed=False) # if network type not specified, then generate random graph (erdos renyi model)

        # self.g = nx.watts_strogatz_graph(n = nodes, k = SW_connections, p = SW_randomness, seed=None)
        self.pos = nx.circular_layout(self.g)
        self.n = nodes
        self.t = 0
        self.p = pandemicprob
        self.p_reduced = reduced_prob # reduced probability of infection due to mitigation
        self.mit_prop = mitigation_proportion # proporation of the population that follows the mitigation
        self.want_plots = plots
        self.want_aspl = aspl
        self.masked = []
        self.sicknodes = [random.randint(0, nodes-1) for i in range(sicknode)] # randomly generate n number of sicknnodes
        # print(self.sicknodes)
        # if not sicknode:
        #     self.sicknodes = set([])
        # else:
        #     self.sicknodes = set([sicknode])
        self.recovered = []
        self.TTR = TTR # days to recover, default is 15, customisable dependent on model

        # initialise TTR
        initial_TTR_vals = [0 if i not in self.sicknodes else TTR + 1 for i in range(nodes)]
        initial_TTR_dict = {i:initial_TTR_vals[i] for i in range(nodes)}
        nx.set_node_attributes(self.g, initial_TTR_dict, name = 'TTR')
    
    def plot(self):

        node_colors = ["green" if node in self.recovered else 'magenta' if node in self.masked else "firebrick" if node in self.sicknodes else "skyblue" for node in self.g.nodes()]
        nx.draw_networkx(self.g, pos = self.pos, with_labels=False, node_size=2000/self.n, node_color=node_colors, linewidths=0.5)

        e_no_mit = [(u, v) for (u, v, d) in self.g.edges(data=True) if d["weight"] == self.p]
        e_mit = [(u, v) for (u, v, d) in self.g.edges(data=True) if d["weight"] == self.p_reduced]
        e_sus = [(u, v) for (u, v, d) in self.g.edges(data=True) if d["weight"] == 0]
        nx.draw_networkx_edges(self.g, self.pos, edgelist=e_no_mit, width=2, edge_color='red') # un-masked
        nx.draw_networkx_edges(self.g, self.pos, edgelist=e_mit, width=2, edge_color='orange') # masked
        nx.draw_networkx_edges(self.g, self.pos, edgelist=e_sus, width=2, edge_color='lime') # susceptible

        plt.title("Small Worlds Graph: Nodes = "+ str(self.n)+ ", Time = " + str(self.t))
        plt.show()
        return
    
    def ASPL(self):
        ASPL_w = []
        for C in (self.g.subgraph(c).copy() for c in nx.connected_components(self.g)):
            ASPL_w.append(nx.average_shortest_path_length(C, weight= 'weight'))
        return statistics.mean(ASPL_w)

    #def sicknode
    def propagate(self):
        #Plot initial network
        # self.plot() 
        timestamps = []
        infectious_count = []
        recovery_count = []
        uninfected_count = []
        cumulative_case_count = []
        aspl_count = []

        # time = 0

        # for time in tqdm.tqdm(range(steps)):
        while len(self.sicknodes) > 0: # whilst there are still sicknodes present
            #check sick nodes
            new_sick = 0
            timestamps.append(self.t)
            # print('Time = ', time)
            # print([self.g.nodes[i]['TTR'] for i in range(len(list(self.g.nodes)))])
            # print('Infectious:', self.sicknodes)
            # print('Recovered:', self.recovered)
            if self.t == 0:
                cumulative_case_count.append(len(self.sicknodes))

                # initialise edge weights
                # for i in self.sicknodes:
                #     if np.random.random() < self.mit_prop: # a proportion of initially sick nodes will adopt mitigation measures
                #         self.masked.append(i)
                #     else: 
                #         pass
                edge_list = [(u, v) for (u, v, d) in self.g.edges(data = True)] # all edges
                actual_m = 0
                for j in edge_list:
                    if np.random.random() < self.mit_prop:
                        self.g.add_edge(j[0], j[1], weight = 1/self.p_reduced) 
                        actual_m += 1
                    else:
                        self.g.add_edge(j[0], j[1], weight = 1/self.p) 
                m_adoption = actual_m / len(edge_list)
                # self.plot()
                # initial_aspl = self.ASPL()
            else: 
                pass
                    
            infectious_count.append(len(self.sicknodes))
            recovery_count.append(len(self.recovered))
            uninfected_count.append(self.n - len(self.recovered) - len(self.sicknodes))
            
            currentsick = copy.copy(self.sicknodes) #as new sick nodes may be created and we don't want to loop through the new ones
            #print("sicknodes:", currentsick)

            for i in range(len(list(self.g.nodes))):
                if self.g.nodes[i]['TTR'] - 1 == 0: # end of infectious period
                    self.sicknodes.remove(i)
                    neighbours = list(self.g.neighbors(i))
                    for n in neighbours:
                        if n in self.sicknodes:
                            pass
                        else:
                            pass
                            # self.g.add_edge(i, n, weight = 1/self.p_reduced) # once recovered, cannot be reinfected again, therefore all the edges should have a weighting of 0.

                    if i in self.masked:
                        self.masked.remove(i)
                    else:
                        pass
                    self.recovered.append(i)
                    nx.set_node_attributes(self.g, {i: self.g.nodes[i]['TTR'] - 1}, name='TTR') # decrement TTR value
                elif self.g.nodes[i]['TTR'] == 0: # not infectious, no action
                    pass
                else: # mid infectious period
                    nx.set_node_attributes(self.g, {i: self.g.nodes[i]['TTR'] - 1}, name='TTR') # decrement TTR value
            
            for node in currentsick:
                neighbours = list(self.g.neighbors(node))

                #try to propagate sickness
                for neighbour in neighbours:
                    if neighbour in self.sicknodes or neighbour in self.recovered: # do not infect again if already sick or recovered.
                        pass
                    elif self.g.get_edge_data(node,neighbour)['weight'] != 0 and np.random.random() < (1/self.g.get_edge_data(node,neighbour)['weight']): # new node infected
                        self.sicknodes.append(neighbour)
                        adj_neighbours = list(self.g.neighbors(neighbour)) # access the neighbours of the newly infected node to create new edge weightings.
                        new_sick += 1

                        nx.set_node_attributes(self.g, {neighbour:self.TTR + 1}, name = 'TTR') # initialise infectious node

                        if np.random.random() < self.mit_prop: # determine whether newly sick node will adopt mitigation measure
                            self.masked.append(neighbour)
                            for i in adj_neighbours:
                                pass
                                # self.g.add_edge(i, neighbour, weight = 1/self.p_reduced) # mitigated probability of infection -> overwrite all edges

                        else: # node did not adopt mitigation measure 
                            for i in adj_neighbours: 
                                if self.g.get_edge_data(i,neighbour)['weight'] == 1/self.p_reduced: # do not overwrite mitigated edges
                                    pass
                                else: 
                                    # self.g.add_edge(i, neighbour, weight = 1/self.p) # non-mitigated probability of infection.
                                    pass

                    else: # node not infected
                        pass
            
            cumulative_case_count.append(cumulative_case_count[-1] + new_sick)
            self.t += 1 #timestep increase by 1
            
            if self.want_plots: 
                self.plot()
            
            if self.want_aspl:
                aspl_count.append(self.ASPL())
            
        

        infected_percentage = cumulative_case_count[-1]/self.n

        if infected_percentage == 1: # entire population infected
            propagation_time = max(timestamps) + 2 - cumulative_case_count.count(cumulative_case_count[-1])
        else:
            propagation_time = 999999999

        if self.want_aspl:
            return timestamps, infectious_count, recovery_count, uninfected_count, cumulative_case_count[:-1], aspl_count
        else:    
            # self.plot()             
            # return timestamps, infectious_count, recovery_count, uninfected_count, cumulative_case_count[:-1]
            return infected_percentage, propagation_time, m_adoption

In [8]:
m_prob_vals = [i/20 for i in range(1,20)]
# m_prob_vals = [0, 0.1, 0.2]
sw = [2, 3, 4, 5]
# sw = [2, 3]
p = 1 # probability of infection
p_reduced = p /2

for c in sw:
    m_results = []
    for m in tqdm.tqdm(m_prob_vals):
        aspl_list = []
        inf_perc_list = []
        prop_time_list = []
        m_adoption_list = []
        repeats = 10
        for _ in range(repeats):
            G = Pandemic_Network(nodes = 2000, network_type='Small World', pandemicprob = p, sicknode = 1, SW_connections = c, SF_k = 2, edge_randomness = 0, 
                                    reduced_prob = p_reduced, mitigation_proportion = m, plots = False, aspl = False)
            inf_perc, prop_time, m_adoption = G.propagate()
            inf_perc_list.append(inf_perc)
            prop_time_list.append(prop_time)
            m_adoption_list.append(m_adoption)
        p_outbreak = (repeats - prop_time_list.count(999999999))/repeats
        aspl = G.ASPL()
        m_results.append([m, aspl, statistics.mean(inf_perc_list), statistics.stdev(inf_perc_list), 
                        statistics.mean(prop_time_list), statistics.stdev(prop_time_list), statistics.mean(m_adoption_list), 
                        statistics.stdev(m_adoption_list), p_outbreak])
        # print(aspl_list, inf_perc_list, prop_time_list)
        # print(inf_perc_list)

    df = pd.DataFrame(m_results)
    df.columns = ['Probability of Adopting Mitigation (Theoretical)', 'Mean ASPL', 'Mean Percentage of Population Infected', 'Std. Dev Percentage of Population Infected', 
                'Mean Propagation Time', 'Std. Dev. Propagation Time', 'Probability of Outbreak', 'Mean of Actual Mitigation Adoption', 'Std. Dev of Actual Mitigation Adoption']
    date = datetime.datetime.today()
    df.to_csv('aspl_m_prob_inf_{}_{}.csv'.format(c, date))


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:20<00:40, 20.15s/it]

[0, 500.25012506253125, 1.0, 0.0, 1000, 0.0, 0.0, 0.0, 1.0]


 67%|██████▋   | 2/3 [00:43<00:22, 22.32s/it]

[0.1, 548.0102816408204, 1.0, 0.0, 1093.5, 10.522462745109731, 0.0974, 0.006919376979018978, 1.0]


 67%|██████▋   | 2/3 [00:52<00:26, 26.02s/it]


KeyboardInterrupt: 